In [13]:
import boto3
from io import StringIO
import pandas as pd
import awswrangler as wr

ModuleNotFoundError: No module named 'awswrangler'

In [ ]:
!pip install awswrangler

In [3]:
import sagemaker
from sagemaker import get_execution_role
from sagemaker.estimator import Estimator
from sagemaker import image_uris

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


In [4]:
# Specify S3 bucket and path
region = sagemaker.Session().boto_region_name
bucket_name = 'viamericas-datalake-dev-us-east-1-283731589572-analytics'
file_name_abt_train = 'json_train_mex.json'
file_name_abt_test = 'json_test_mex.json'

path_abt_train = f's3://{bucket_name}/Forecast/data_full/{file_name_abt_train}'
path_abt_test = f's3://{bucket_name}/Forecast/data_full/{file_name_abt_test}'
output_s3_path = 's3://viamericas-datalake-dev-us-east-1-283731589572-analytics/Forecast/data_full/'

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


In [5]:
#Set up SageMaker session and role
sagemaker_session = sagemaker.Session()
#role = 'arn:aws:iam::283731589572:role/service-role/AmazonForecast-ExecutionRole-1704319864386'
role = 'arn:aws:iam::283731589572:role/service-role/AmazonSageMaker-ExecutionRole-20231127T122365'
#role = sagemaker.get_execution_role()
role

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


'arn:aws:iam::283731589572:role/service-role/AmazonSageMaker-ExecutionRole-20231127T122365'

In [6]:
# Cargar el JSON desde S3
json_train_s3_url = f's3://{bucket_name}/Forecast/data_full/{file_name_abt_train}'
json_train_s3_url

's3://viamericas-datalake-dev-us-east-1-283731589572-analytics/Forecast/data_full/json_train_mex.json'

In [ ]:
json_train_data = pd.read_json(json_train_s3_url)

In [ ]:
# Muestra del conjunto de datos p/extraer las features dinámicas
sample_data = json_train_data.head()
dynamic_feat_sample = sample_data['dynamic_feat'].iloc[0]

# Número de features dinámicas
num_dynamic_feat = len(dynamic_feat_sample)
num_dynamic_feat

In [ ]:
# Calcular la cardinalidad de 'payer_country'
cardinality_payer_country = json_train_data['cat'].apply(lambda x: x['payer_country']).nunique()
cardinality_payer_country

In [ ]:
# URI Image for AMZ DeepAR
image_name = sagemaker.image_uris.retrieve("forecasting-deepar", region)
image_name

In [53]:
# Especificar la imagen de contenedor de DeepAR
container = sagemaker.image_uris.retrieve("forecasting-deepar", region, version="1")
container

INFO:sagemaker.image_uris:Same images used for training and inference. Defaulting to image scope: inference.
INFO:sagemaker.image_uris:Ignoring unnecessary instance type: None.


'522234722520.dkr.ecr.us-east-1.amazonaws.com/forecasting-deepar:1'

In [54]:
# Crear el estimador DeepAR
estimator = Estimator(
    container,
    role=role,
    instance_count=1,
    instance_type='ml.m4.xlarge',  # O selecciona el tipo de instancia que desees
    output_path=output_s3_path,
    sagemaker_session=sagemaker_session
)

In [55]:
# Definir hiperparámetros del modelo
hyperparameters = {
    "time_freq": "D",
    "epochs": 10,
    "context_length": 30,
    "prediction_length": 10
}

In [56]:
# Establecer los hiperparámetros en el estimador
estimator.set_hyperparameters(**hyperparameters)


In [57]:
# Entrenar el modelo con los datos de entrenamiento en S3
estimator.fit({'train': path_abt_train})

INFO:sagemaker:Creating training-job with name: forecasting-deepar-2024-02-05-20-28-53-212


2024-02-05 20:28:53 Starting - Starting the training job...
2024-02-05 20:29:19 Starting - Preparing the instances for training............
2024-02-05 20:31:03 Downloading - Downloading input data...
2024-02-05 20:31:33 Downloading - Downloading the training image............
2024-02-05 20:33:54 Training - Training image download completed. Training in progress....
2024-02-05 20:34:20 Uploading - Uploading generated training modelDocker entrypoint called with argument(s): train
Running default environment configuration script
Running custom environment configuration script
/opt/amazon/lib/python3.8/site-packages/mxnet/model.py:97: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if num_device is 1 and 'dist' not in kvstore:
[02/05/2024 20:34:16 INFO 140509669033792] Reading default configuration from /opt/amazon/lib/python3.8/site-packages/algorithm/resources/default-input.json: {'_kvstore': 'auto', '_num_gpus': 'auto', '_num_kv_servers': 'auto', '_tuning_objective_metric': '',

UnexpectedStatusException: Error for Training job forecasting-deepar-2024-02-05-20-28-53-212: Failed. Reason: ClientError: Error when loading json (source: /opt/ml/input/data/train/json_train_mex.json, row: 1): line should contain an object, but contains type <class 'list'>., exit code: 2